In [1]:
import logging
import os
import glob
from pathlib import Path
import re
from speechbrain.dataio.dataio import load_pkl, merge_csvs, save_pkl
# from speechbrain.utils.data_utils import get_all_files

## Kang
import pandas as pd
import json
import os
from pydub import AudioSegment
from datetime import datetime

In [2]:
logger = logging.getLogger(__name__)

In [3]:
def check_kdialectspeech_folders(base_dir, province_code):
    """
    Check if the data folder actually contains the kdialectspeech dataset.

    If it does not, an error is raised.

    Returns
    -------
    None

    Raises
    ------
    OSError
        If kdialectspeech is not found at the specified path.
    -----
    ex) check_kdialectspeech_folders("/data/aidata", "gw")
    """
    # /data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도
    # /data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 경상도
    # /data/aidata/139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 충청도
    # /data/aidata/139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 전라도
    # /data/aidata/139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/03. 제주도

    if province_code == "gw":
        label_dir = os.path.join(base_dir, "139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도")
    elif province_code == "gs":
        label_dir = os.path.join(base_dir, "139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 경상도")
    elif province_code == "cc":
        label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 충청도")
    elif province_code == "jl":
        label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 전라도")
    elif province_code == "jj":
        label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/03. 제주도")
    else:
        err_msg = (
            "지역 코드를 gw, gs, cc, jl, jj 중 하나를 입력해야 합니다. (gw:강원도, gs:경상도, cc:충청도, jl:전라도, jj:제주도)"
        )
        raise OSError(err_msg)

    if not os.path.exists(label_dir):
        err_msg = (
            "the directory %s does not exist (it is expected in the "
            "kdialectspeech dataset)" % label_dir
        )
        raise OSError(err_msg)
    else:
        return True

In [11]:
base_dir = "/data/aidata"
province_code = "gw"

In [12]:
if province_code == "gw":
    label_dir = os.path.join(base_dir, "139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도")
elif province_code == "gs":
    label_dir = os.path.join(base_dir, "139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 경상도")
elif province_code == "cc":
    label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 충청도")
elif province_code == "jl":
    label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/02. 전라도")
elif province_code == "jj":
    label_dir = os.path.join(base_dir, "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)/06.품질검증/1.Dataset/2.라벨링데이터/03. 제주도")
else:
    err_msg = (
        "지역 코드를 gw, gs, cc, jl, jj 중 하나를 입력해야 합니다. (gw:강원도, gs:경상도, cc:충청도, jl:전라도, jj:제주도)"
    )
    raise OSError(err_msg)
label_dir

'/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도'

In [13]:
json_file_list = (glob.glob(os.path.join(label_dir, '*/*/*.' + "json")))
# json_file_list

['/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_14_0_31.json',
 '/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_13_0_27.json',
 '/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_1_0_22.json',
 '/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_9_0_22.json',
 '/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_1_1_30.json',
 '/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459

In [14]:
json_file_path = "/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/2.라벨링데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_14_0_31.json"
data_file_path = json_file_path.replace("2.라벨링데이터", "1.원천데이터")
data_file_path

'/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/03. 2인발화/collectorgw99/talk_set1_collectorgw99_speakergw459_speakergw460_14_0_31.json'

In [15]:
os.path.dirname(data_file_path)

'/data/aidata/139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/03. 2인발화/collectorgw99'

In [16]:
Path(json_file_path).stem

'talk_set1_collectorgw99_speakergw459_speakergw460_14_0_31'